# PROJET D'OPTIMISATION - GRP 4 

## Récupération de données 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Question 1

### Question 4

In [ ]:
def grad(u):
    """Returns the discrete gradient of matrix u."""
    (n,m) = u.shape
    
    #grad_x
    grad_x = u[1:n-1,:]-u[0:n-2,:]
    grad_x = np.append(grad_x,np.zeros((1,m),dtype=float),axis=0) # zeros for line i=n

    #grad_y
    grad_y = u[:,1:m-1]-u[:,0:m-2]
    grad_y = np.append(grad_y,np.zeros((n,1),dtype=float),axis=1) # zeros for column j=m

    return(grad_x, grad_y)